In [1]:
import math
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import sqlite3

In [2]:
def tile_to_lat_lon(zoom, x, y):
    n = 2 ** zoom
    lon = x / n * 360.0 - 180.0
    lat = math.degrees(math.atan(math.sinh(math.pi * (1 - 2 * y / n))))
    return lat, lon

def tile_to_polygon(zoom, x, y):
    lat1, lon1 = tile_to_lat_lon(zoom, x, y)  # Top-left corner
    lat2, lon2 = tile_to_lat_lon(zoom, x + 1, y + 1)  # Bottom-right corner
    return Polygon([(lon1, lat1), (lon2, lat1), (lon2, lat2), (lon1, lat2)])

def lat_lon_to_tile(lat, lon, zoom):
    n = 2 ** zoom
    x = int((lon + 180.0) / 360.0 * n)
    y = int((1 - math.log(math.tan(math.radians(lat)) + 1 / math.cos(math.radians(lat))) / math.pi) / 2 * n)
    return x, y

def generate_tiles_from_top_left(lat, lon, zoom, n, m):
    top_left_x, top_left_y = lat_lon_to_tile(lat, lon, zoom)

    tiles = []
    for i in range(m):
        for j in range(n):
            x = top_left_x + i
            y = top_left_y + j
            x_rel = i
            y_rel = j
            tiles.append((zoom, x, y, x_rel, y_rel))

    return tiles

def generate_geodataframe_for_tiles(tiles):
    records = []
    for zoom, x, y, x_rel, y_rel in tiles:
        polygon = tile_to_polygon(zoom, x, y)
        records.append({
            'zoom': zoom,
            'x': x,
            'y': y,
            'x_rel': x_rel,
            'y_rel': y_rel,
            'geometry': polygon
        })

    gdf = gpd.GeoDataFrame(records, crs="EPSG:4326")
    return gdf


In [3]:
ref_lat, ref_lon = 35.42514145560098, 139.58607122847198
zoom = 17
n,m = 4,5 # vertical, horizontal
tiles = generate_tiles_from_top_left(ref_lat,ref_lon,zoom,n,m)
gdf = generate_geodataframe_for_tiles(tiles)
gdf["label"] = 0 # ラベル用

In [4]:
# Create a Folium map centered on the specified coordinates

m = folium.Map(location=[ref_lat, ref_lon], zoom_start=14)

# Add GeoDataFrame polygons to the map
for _, row in gdf.iterrows():
    # Extract the polygon and convert it to a format that folium understands (GeoJSON)
    folium.GeoJson(row['geometry'],
                   style_function=lambda x: {
                       'fillColor': 'blue',
                       'color': 'blue',
                       'weight': 1,
                       'fillOpacity': 0.2
                   }).add_to(m)

# Display the map
m


In [4]:
gdf

,zoom,x,y,x_rel,y_rel,geometry,label
0,17,116358,51733,0,0,"POLYGON ((139.58679 35.41368, 139.58954 35.413...",0
1,17,116358,51734,0,1,"POLYGON ((139.58679 35.41144, 139.58954 35.411...",0
2,17,116358,51735,0,2,"POLYGON ((139.58679 35.4092, 139.58954 35.4092...",0
3,17,116358,51736,0,3,"POLYGON ((139.58679 35.40696, 139.58954 35.406...",0
4,17,116359,51733,1,0,"POLYGON ((139.58954 35.41368, 139.59229 35.413...",0
5,17,116359,51734,1,1,"POLYGON ((139.58954 35.41144, 139.59229 35.411...",0
6,17,116359,51735,1,2,"POLYGON ((139.58954 35.4092, 139.59229 35.4092...",0
7,17,116359,51736,1,3,"POLYGON ((139.58954 35.40696, 139.59229 35.406...",0
8,17,116360,51733,2,0,"POLYGON ((139.59229 35.41368, 139.59503 35.413...",0
9,17,116360,51734,2,1,"POLYGON ((139.59229 35.41144, 139.59503 35.411...",0


In [5]:
path_to_db = f"C:\\Users\\tora2\\IdeaProjects\\cityScope\\visual\\tangible_ui_01\\tile.db"

In [6]:
conn = sqlite3.connect(path_to_db)
cursor = conn.cursor()
cursor.execute("DROP TABLE tile")
df = gdf.copy()
df["geometry"] = df["geometry"].astype(str)
df.to_sql('tile', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

C:\Users\tora2\AppData\Local\Temp\ipykernel_12344\3189415618.py:5: UserWarning: Geometry column does not contain geometry.
  df["geometry"] = df["geometry"].astype(str)


In [7]:
conn = sqlite3.connect(path_to_db)
cursor = conn.cursor()
cursor.execute("SELECT * FROM tile")
for r in cursor.fetchall():
    print(r)
conn.close()

(17, 116357, 51727, 0, 0, 'POLYGON ((139.584045 35.427106, 139.586792 35.427106, 139.586792 35.424868, 139.584045 35.424868, 139.584045 35.427106))', 0)
(17, 116357, 51728, 0, 1, 'POLYGON ((139.584045 35.424868, 139.586792 35.424868, 139.586792 35.42263, 139.584045 35.42263, 139.584045 35.424868))', 0)
(17, 116357, 51729, 0, 2, 'POLYGON ((139.584045 35.42263, 139.586792 35.42263, 139.586792 35.420392, 139.584045 35.420392, 139.584045 35.42263))', 0)
(17, 116357, 51730, 0, 3, 'POLYGON ((139.584045 35.420392, 139.586792 35.420392, 139.586792 35.418153, 139.584045 35.418153, 139.584045 35.420392))', 0)
(17, 116358, 51727, 1, 0, 'POLYGON ((139.586792 35.427106, 139.589539 35.427106, 139.589539 35.424868, 139.586792 35.424868, 139.586792 35.427106))', 0)
(17, 116358, 51728, 1, 1, 'POLYGON ((139.586792 35.424868, 139.589539 35.424868, 139.589539 35.42263, 139.586792 35.42263, 139.586792 35.424868))', 0)
(17, 116358, 51729, 1, 2, 'POLYGON ((139.586792 35.42263, 139.589539 35.42263, 139.589539